## Import

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import *

## Spark Session

In [2]:
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BatchDataProcessing")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")


## Create the Session

In [3]:
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [4]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

## Loading Data from BigQuery

In [5]:
flightDf = spark.read \
  .format("bigquery") \
  .load("data-eng-assignment-2.assignment2.flightdata")

### Show Data

In [6]:
flightDf.printSchema()
flightDf.show(4)

root
 |-- FL_DATE: date (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- DOT_CODE: long (nullable = true)
 |-- FL_NUMBER: long (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY: string (nullable = true)
 |-- CRS_DEP_TIME: long (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: long (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DIST

## Drop Irrelevant Columns

In [7]:
flightDf = flightDf.drop("DELAY_DUE_CARRIER","DELAY_DUE_WEATHER","DELAY_DUE_NAS","DELAY_DUE_SECURITY","DELAY_DUE_LATE_AIRCRAFT","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN")
flightDf.printSchema()
flightDf.show(10)

root
 |-- FL_DATE: date (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- DOT_CODE: long (nullable = true)
 |-- FL_NUMBER: long (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY: string (nullable = true)
 |-- CRS_DEP_TIME: long (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- CRS_ARR_TIME: long (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- FL_YEAR: long (nullable = true)
 |-- FL_MONTH: long (nullable = true)
 |-- FL_DAY: long (nullable = true)

+----------+-------

In [8]:
flightDfProcessed = flightDf.groupBy('FL_DATE').agg(count('*').alias('number_of_flights'))

In [9]:
flightDfProcessed.printSchema()
flightDfProcessed = flightDfProcessed.orderBy(col('FL_DATE').asc())
flightDfProcessed.show(10)

root
 |-- FL_DATE: date (nullable = true)
 |-- number_of_flights: long (nullable = false)

+----------+-----------------+
|   FL_DATE|number_of_flights|
+----------+-----------------+
|2020-01-01|            18114|
|2020-01-02|            20612|
|2020-01-03|            20616|
|2020-01-04|            18099|
|2020-01-05|            20632|
|2020-01-06|            20943|
|2020-01-07|            19164|
|2020-01-08|            19670|
|2020-01-09|            20789|
|2020-01-10|            20810|
+----------+-----------------+
only showing top 10 rows



In [10]:
averageDelayDf = flightDf.groupBy('FL_DATE').agg(avg('DEP_DELAY').alias('AVG_DELAY_DAY'))
averageDelayDf.show(10)

+----------+-------------------+
|   FL_DATE|      AVG_DELAY_DAY|
+----------+-------------------+
|2020-08-24|-1.0528630838131798|
|2020-07-24|-0.2810010461092782|
|2020-08-05|-1.9180387190157409|
|2020-01-21| 3.4156082712291385|
|2020-11-29| 0.3615639496355202|
|2020-08-28|  2.814814814814815|
|2020-08-29|-2.0208619505494507|
|2020-04-30| -2.963560447902828|
|2020-10-04| 0.5998155445627568|
|2020-09-24|-1.1462092258465915|
+----------+-------------------+
only showing top 10 rows



In [11]:
flightDfProcessed = flightDfProcessed.join(averageDelayDf, 'FL_DATE', 'left')

In [12]:
flightDfProcessed = flightDfProcessed.orderBy(col('FL_DATE').asc())
flightDfProcessed.show(10)


+----------+-----------------+------------------+
|   FL_DATE|number_of_flights|     AVG_DELAY_DAY|
+----------+-----------------+------------------+
|2020-01-01|            18114| 5.649816687034774|
|2020-01-02|            20612| 6.620170523751523|
|2020-01-03|            20616| 8.703171432747114|
|2020-01-04|            18099|13.947735964085796|
|2020-01-05|            20632| 9.104095729156533|
|2020-01-06|            20943| 5.722735320859454|
|2020-01-07|            19164|2.0051338467180053|
|2020-01-08|            19670| 4.033389493031092|
|2020-01-09|            20789|1.1570271833429728|
|2020-01-10|            20810| 8.077751733080644|
+----------+-----------------+------------------+
only showing top 10 rows



In [13]:
flightDfProcessed = flightDfProcessed.withColumn('AVG_DELAY_DAY', round(col('AVG_DELAY_DAY'),1))

In [14]:
bucket = "temp_bucket_prj2"
spark.conf.set('temporaryGcsBucket', bucket)
# Saving the data to BigQuery
flightDfProcessed.write.format('bigquery') \
  .option('table', 'data-eng-assignment-2.assignment2.avg-delay-per-day') \
  .option('temporaryGcsBucket', bucket) \
  .mode('overwrite') \
  .save()

In [15]:
weatherDf = spark.read \
  .format("bigquery") \
  .load("data-eng-assignment-2.assignment2.weatherdata")

In [16]:
weatherDf.printSchema()
weatherDf.show(10)

root
 |-- FL_DATE: date (nullable = true)
 |-- WIND_SPEED: double (nullable = true)

+----------+----------+
|   FL_DATE|WIND_SPEED|
+----------+----------+
|2020-03-14|       3.5|
|2020-12-14|       3.5|
|2020-10-28|       4.0|
|2020-07-06|       7.5|
|2020-01-24|       8.0|
|2020-08-12|       8.0|
|2020-12-05|       8.0|
|2020-08-03|       9.0|
|2020-11-11|       9.0|
|2020-10-26|       9.5|
+----------+----------+
only showing top 10 rows



In [17]:
weatherDf = weatherDf.orderBy(col('FL_DATE').asc())

In [18]:
weatherDf.show(10)
weatherDf.count()

+----------+----------+
|   FL_DATE|WIND_SPEED|
+----------+----------+
|2020-01-01|      49.7|
|2020-01-02|       5.9|
|2020-01-03|      22.1|
|2020-01-04|      46.4|
|2020-01-05|      45.4|
|2020-01-06|       7.1|
|2020-01-07|      14.0|
|2020-01-08|      48.0|
|2020-01-09|      45.1|
|2020-01-10|      29.3|
+----------+----------+
only showing top 10 rows



366

In [19]:
weatherDelay = flightDfProcessed.join(weatherDf, 'FL_DATE', 'left').orderBy(col('FL_DATE').asc())
weatherDelay.show(10)


+----------+-----------------+-------------+----------+
|   FL_DATE|number_of_flights|AVG_DELAY_DAY|WIND_SPEED|
+----------+-----------------+-------------+----------+
|2020-01-01|            18114|          5.6|      49.7|
|2020-01-02|            20612|          6.6|       5.9|
|2020-01-03|            20616|          8.7|      22.1|
|2020-01-04|            18099|         13.9|      46.4|
|2020-01-05|            20632|          9.1|      45.4|
|2020-01-06|            20943|          5.7|       7.1|
|2020-01-07|            19164|          2.0|      14.0|
|2020-01-08|            19670|          4.0|      48.0|
|2020-01-09|            20789|          1.2|      45.1|
|2020-01-10|            20810|          8.1|      29.3|
+----------+-----------------+-------------+----------+
only showing top 10 rows



In [20]:
weatherDelay = weatherDelay.withColumn('AVG_DELAY_DAY', round(col('AVG_DELAY_DAY'),1))
weatherDelay.show(10)

+----------+-----------------+-------------+----------+
|   FL_DATE|number_of_flights|AVG_DELAY_DAY|WIND_SPEED|
+----------+-----------------+-------------+----------+
|2020-01-01|            18114|          5.6|      49.7|
|2020-01-02|            20612|          6.6|       5.9|
|2020-01-03|            20616|          8.7|      22.1|
|2020-01-04|            18099|         13.9|      46.4|
|2020-01-05|            20632|          9.1|      45.4|
|2020-01-06|            20943|          5.7|       7.1|
|2020-01-07|            19164|          2.0|      14.0|
|2020-01-08|            19670|          4.0|      48.0|
|2020-01-09|            20789|          1.2|      45.1|
|2020-01-10|            20810|          8.1|      29.3|
+----------+-----------------+-------------+----------+
only showing top 10 rows



In [21]:
weatherDelay.write.format('bigquery') \
  .option('table', 'data-eng-assignment-2.assignment2.weather-delay') \
  .option('temporaryGcsBucket', bucket) \
  .mode('overwrite') \
  .save()

In [22]:
tempDf = flightDf.groupBy('AIRLINE_CODE').agg(avg('DEP_DELAY').alias('AVG_DELAY_AIRLINE'))
tempDf.show()

+------------+--------------------+
|AIRLINE_CODE|   AVG_DELAY_AIRLINE|
+------------+--------------------+
|          UA|  1.9604145810435234|
|          NK|  3.8635644837892027|
|          AA|   3.062314246456681|
|          EV|-0.04897721603193...|
|          B6|  1.4670518448911272|
|          DL|  1.5880461701110806|
|          OO|   3.782471560626977|
|          F9|   0.904193347049169|
|          YV|   4.770665208390297|
|          MQ|  1.6796341073498262|
|          OH|   4.839221922349015|
|          HA| -1.2147764976356559|
|          G4|   5.907015960478814|
|          YX|   -0.54213591861893|
|          AS| -1.1810845804101051|
|          WN|  1.0216153510980834|
|          9E|  0.7669428115922183|
+------------+--------------------+



In [23]:
airlineCodeDictDf = spark.read \
  .format("bigquery") \
  .load("data-eng-assignment-2.assignment2.airlinecodes")

In [24]:
delayByAirlineDf = tempDf.join(airlineCodeDictDf, airlineCodeDictDf['CODE']==tempDf['AIRLINE_CODE'], how='left')
delayByAirlineDf = delayByAirlineDf.orderBy(col('AVG_DELAY_AIRLINE').desc())
delayByAirlineDf.show()


+------------+--------------------+----+--------------------+
|AIRLINE_CODE|   AVG_DELAY_AIRLINE|CODE|               DESCR|
+------------+--------------------+----+--------------------+
|          G4|   5.907015960478814|  G4|       Allegiant Air|
|          OH|   4.839221922349015|  OH|   PSA Airlines Inc.|
|          YV|   4.770665208390297|  YV|  Mesa Airlines Inc.|
|          NK|  3.8635644837892027|  NK|    Spirit Air Lines|
|          OO|   3.782471560626977|  OO|SkyWest Airlines ...|
|          AA|   3.062314246456681|  AA|American Airlines...|
|          UA|  1.9604145810435234|  UA|United Air Lines ...|
|          MQ|  1.6796341073498262|  MQ|           Envoy Air|
|          DL|  1.5880461701110806|  DL|Delta Air Lines Inc.|
|          B6|  1.4670518448911272|  B6|     JetBlue Airways|
|          WN|  1.0216153510980834|  WN|Southwest Airline...|
|          F9|   0.904193347049169|  F9|Frontier Airlines...|
|          9E|  0.7669428115922183|  9E|   Endeavor Air Inc.|
|       

In [25]:
delayByAirlineDf=delayByAirlineDf.drop('CODE')
delayByAirlineDf.show()

+------------+--------------------+--------------------+
|AIRLINE_CODE|   AVG_DELAY_AIRLINE|               DESCR|
+------------+--------------------+--------------------+
|          G4|   5.907015960478814|       Allegiant Air|
|          OH|   4.839221922349015|   PSA Airlines Inc.|
|          YV|   4.770665208390297|  Mesa Airlines Inc.|
|          NK|  3.8635644837892027|    Spirit Air Lines|
|          OO|   3.782471560626977|SkyWest Airlines ...|
|          AA|   3.062314246456681|American Airlines...|
|          UA|  1.9604145810435234|United Air Lines ...|
|          MQ|  1.6796341073498262|           Envoy Air|
|          DL|  1.5880461701110806|Delta Air Lines Inc.|
|          B6|  1.4670518448911272|     JetBlue Airways|
|          WN|  1.0216153510980834|Southwest Airline...|
|          F9|   0.904193347049169|Frontier Airlines...|
|          9E|  0.7669428115922183|   Endeavor Air Inc.|
|          EV|-0.04897721603193...|ExpressJet Airlin...|
|          YX|   -0.54213591861

In [44]:
delayByAirlineDf.write.format('bigquery') \
  .option('table', 'data-eng-assignment-2.assignment2.airline-delay') \
  .option('temporaryGcsBucket', bucket) \
  .mode('overwrite') \
  .save()

In [26]:
avgDelayOrg = flightDf.groupBy('ORIGIN').agg(avg('DEP_DELAY').alias('AVG_DELAY_ORIGIN'))
avgDelayOrg.show()

+------+--------------------+
|ORIGIN|    AVG_DELAY_ORIGIN|
+------+--------------------+
|   BGM|   16.27251184834123|
|   DLG| -0.9568345323741008|
|   PSE|  7.2405063291139244|
|   INL|   5.271626297577854|
|   MSY|   3.410666401827391|
|   PPG|   80.04347826086956|
|   GEG|  0.4766497946709961|
|   DRT|   6.505938242280285|
|   SNA|  0.6287617839013778|
|   BUR|-0.14614284826257226|
|   GRB|   1.483450031826862|
|   GTF|   5.733113673805601|
|   IDA|   3.748644652538196|
|   GRR|   2.731504366452463|
|   LWB|    4.54014598540146|
|   PVU|   5.418685121107266|
|   JLN|                 6.6|
|   PSG| -3.8025387870239773|
|   EUG|  2.2387505460899955|
|   ATY| -0.3703071672354949|
+------+--------------------+
only showing top 20 rows



In [27]:
avgDelayOrg = avgDelayOrg.orderBy(col('AVG_DELAY_ORIGIN').desc())

avgDelayOrg.show(4)

+------+------------------+
|ORIGIN|  AVG_DELAY_ORIGIN|
+------+------------------+
|   PPG| 80.04347826086956|
|   CDB|31.733333333333334|
|   ALS|             31.52|
|   MMH|23.597989949748744|
+------+------------------+
only showing top 4 rows



In [45]:
avgDelayOrg.write.format('bigquery') \
  .option('table', 'data-eng-assignment-2.assignment2.origin-delay') \
  .option('temporaryGcsBucket', bucket) \
  .mode('overwrite') \
  .save()

In [29]:
from pyspark.ml.feature import StringIndexer

In [35]:
indexerOrg = StringIndexer(inputCol = 'ORIGIN', outputCol = 'ORIGIN_FACTOR')
indexedDfOrg = indexer.fit(avgDelayOrg).transform(avgDelayOrg)

In [36]:
indexedDfOrg.printSchema()
indexedDfOrg.orderBy(col('ORIGIN_FACTOR').asc()).show()

root
 |-- ORIGIN: string (nullable = true)
 |-- AVG_DELAY_ORIGIN: double (nullable = true)
 |-- ORIGIN_FACTOR: double (nullable = false)

+------+-------------------+-------------+
|ORIGIN|   AVG_DELAY_ORIGIN|ORIGIN_FACTOR|
+------+-------------------+-------------+
|   ABE|  7.739580068943905|          0.0|
|   ABI|  1.585247883917775|          1.0|
|   ABQ| 0.8052434456928839|          2.0|
|   ABR|  8.284457478005866|          3.0|
|   ABY|  1.153225806451613|          4.0|
|   ACK|-1.1297169811320755|          5.0|
|   ACT|0.28475551294343243|          6.0|
|   ACV|   9.92088888888889|          7.0|
|   ACY|-0.9722076560041951|          8.0|
|   ADK| 10.313131313131313|          9.0|
|   ADQ|-2.9079229122055676|         10.0|
|   AEX| 3.7857793983591614|         11.0|
|   AGS| 3.4020100502512562|         12.0|
|   AKN| 2.0552763819095476|         13.0|
|   ALB| 1.3963246997540153|         14.0|
|   ALO|  1.696551724137931|         15.0|
|   ALS|              31.52|         16.0|
| 

In [37]:
correlationOrigin = indexedDfOrg.corr('AVG_DELAY_ORIGIN','ORIGIN_FACTOR')
correlationOrigin

-0.05097058528947979

In [40]:
indexerAirline = StringIndexer(inputCol = 'AIRLINE_CODE', outputCol = 'AIRLINE_CODE_F')
indexedDfAirline = indexerAirline.fit(delayByAirlineDf).transform(delayByAirlineDf)
indexedDfAirline.orderBy(col('AIRLINE_CODE_F').asc()).show()

+------------+--------------------+--------------------+--------------+
|AIRLINE_CODE|   AVG_DELAY_AIRLINE|               DESCR|AIRLINE_CODE_F|
+------------+--------------------+--------------------+--------------+
|          9E|  0.7669428115922183|   Endeavor Air Inc.|           0.0|
|          AA|   3.062314246456681|American Airlines...|           1.0|
|          AS| -1.1810845804101051|Alaska Airlines Inc.|           2.0|
|          B6|  1.4670518448911272|     JetBlue Airways|           3.0|
|          DL|  1.5880461701110806|Delta Air Lines Inc.|           4.0|
|          EV|-0.04897721603193...|ExpressJet Airlin...|           5.0|
|          F9|   0.904193347049169|Frontier Airlines...|           6.0|
|          G4|   5.907015960478814|       Allegiant Air|           7.0|
|          HA| -1.2147764976356559|Hawaiian Airlines...|           8.0|
|          MQ|  1.6796341073498262|           Envoy Air|           9.0|
|          NK|  3.8635644837892027|    Spirit Air Lines|        

In [41]:
corrAirline = indexedDfAirline.corr('AVG_DELAY_AIRLINE', 'AIRLINE_CODE_F')
corrAirline

0.24342065643270153

In [28]:
correlation_result = weatherDelay.corr('WIND_SPEED', 'AVG_DELAY_DAY')
correlation_result

-0.028922234766114083

In [42]:
corrDf = spark.createDataFrame(
    [
        ('ORIGIN',correlationOrigin),
        ('AIRLINE', corrAirline),
        ('WEATHER', correlation_result)
    ],
    ['CORR_VAR', 'CORR_VAL']
)

In [43]:
corrDf.printSchema()
corrDf.show()

root
 |-- CORR_VAR: string (nullable = true)
 |-- CORR_VAL: double (nullable = true)

+--------+--------------------+
|CORR_VAR|            CORR_VAL|
+--------+--------------------+
|  ORIGIN|-0.05097058528947979|
| AIRLINE| 0.24342065643270153|
| WEATHER|-0.02892223476611...|
+--------+--------------------+

